# PREPARATION

## Import Required Library

In [64]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import json
from pprint import pprint

## Global Helper Function

In [71]:
def create_criteria_decision_form(total_criteria: int) -> None: 
    global criterias
    
    dropdowns = []
    for i in range(total_criteria):
        label = widgets.Label(value=f"C{i+1}", layout=widgets.Layout(width="50px"))
    
        dropdown = widgets.Dropdown(options=['Benefit', 'Cost'], layout=widgets.Layout(width="100px"))
        dropdowns.append(dropdown)
    
        display(widgets.HBox([label, dropdown]))

    output = widgets.Output()
    
    def set_criteria(_):
        for (key, dropdown) in zip(criterias.keys(), dropdowns):
            criterias[key]["type"] = dropdown.value
            
        with output:
            output.clear_output(wait=True)
            print('You successfully to set the criteria')
            print(json.dumps(criterias, indent=4))
        
    set_criteria_btn = widgets.Button(description="Set Criteria!")
    set_criteria_btn.on_click(set_criteria)

    display(set_criteria_btn, output)

In [72]:
def create_matrix_form(rows: int, cols: int) -> None:
    col_labels = [""] + [f"C{i+1}" for i in range(cols)]
    row_labels = [f"A{i+1}" for i in range(rows)]
    
    inputs = [[widgets.FloatText(value=0, layout=widgets.Layout(width="50px")) 
               for c in range(cols)] for r in range(rows)]

    header_row = [widgets.Label(value=label, layout=widgets.Layout(width="50px")) for label in col_labels]
    
    input_rows = []
    for r in range(rows):
        row_widgets = [widgets.Label(value=row_labels[r], layout=widgets.Layout(width="50px"))] + inputs[r]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return np.array([[cell.value for cell in row] for row in inputs])

    output = widgets.Output()
    
    def update_matrix(_):
        global decision_matrix
        decision_matrix = get_matrix()

        with output:
            print("Decision matrix:")
            pprint(decision_matrix)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    display(widgets.HBox(header_row))
    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

---

# Input Starter Matrix-Like Data Structure of Criterias and Alternatives

In [32]:
total_criteria = 5
total_alternative = 5

Define criterias

In [57]:
criterias = {f"C{i+1}": {'type': None} for i in range(total_criteria)}

print(json.dumps(criterias, indent=4))

{
    "C1": {
        "type": null
    },
    "C2": {
        "type": null
    },
    "C3": {
        "type": null
    },
    "C4": {
        "type": null
    },
    "C5": {
        "type": null
    }
}


Decide which criteria is benefit or cost

In [62]:
create_criteria_decision_form(total_criteria)

Button(description='Set Criteria!', style=ButtonStyle())

Output()

Create matrix for decision

In [73]:
decision_matrix = [[0 for j in range(total_criteria)] for i in range(total_alternative)]

pprint(decision_matrix)

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]


In [74]:
create_matrix_form(rows=total_alternative, cols=total_criteria)

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Normalization

In [40]:
print('Normalization should be here')

Normalization should be here


# Calculate Normalization Matrix by Weights

In [41]:
print('Calculation should be here')

Calculation should be here


# Get Concordance and Discordance Set

## Concordance

In [42]:
print('Calculation should be here')

Calculation should be here


## Discordance

In [43]:
print('Calculation should be here')

Calculation should be here


# Determine the Dominant Aggregate of The Matrix

In [44]:
print('Calculation should be here')

Calculation should be here


# Eliminate Less Favourable Alternatives

In [45]:
print('Calculation should be here')

Calculation should be here


# Rangking Alternative

In [46]:
print('Calculation should be here')

Calculation should be here


---

# Conclusion